In [1]:

# Import statement

import json

from typing import List, Dict

import numpy as np

import pandas as pd

from keras.models import Model, Input
from keras.layers import LSTM, Dense, Concatenate
from keras.losses import categorical_crossentropy
from keras.activations import sigmoid, softmax, relu, tanh
from keras.metrics import Accuracy
from keras.optimizers import Adam

from keras.preprocessing.sequence import pad_sequences

import pickle



Using TensorFlow backend.


In [2]:


def __import_data_dense__(filename: str):
        # Read json or dat
    
    with open(filename, 'r') as f:
        obj = json.load(f)

    X_dense = np.empty(shape=(0, 208))
    Y_dense = np.empty(shape=(0, 52))

    for match in obj['X']:
        for turn in match:
            turn_vector = np.empty(shape=(0,))
            for feature in turn:
                for card in feature:
                    turn_vector = np.append(turn_vector, [card], axis=0)
            self.X_dense = np.append(self.X_dense, [turn_vector], axis=0)

    for match in obj['Y']:
        for turn in match:
            turn_vector = np.empty(shape=(0,))
            for card in feature:
                turn_vector = np.append(turn_vector, [card], axis=0)
            self.Y_dense = np.append(self.Y_dense, [turn_vector], axis=0)

    return X_dense, Y_dense


def __import_data_lstm__(filename: str):
    # Read json or dat
    
    with open(filename, 'r') as f:
        obj = json.load(f)

    # Set var
    match_list_X_f0 = []
    match_list_X_f1 = []
    match_list_X_f2 = []
    match_list_X_f3 = []

    match_list_Y = []

    for match in obj['X']:
        turn_list = np.empty(shape=(0,52))
        for turn in match:
            card_list = np.empty(shape=(0,))
            for card in turn[0]:
                card_list = np.append(card_list, [card], axis=0)
            turn_list = np.append(turn_list, [card_list], axis=0)
        match_list_X_f0.append(turn_list)

    for match in obj['X']:
        turn_list = np.empty(shape=(0,52))
        for turn in match:
            card_list = np.empty(shape=(0,))
            for card in turn[1]:
                card_list = np.append(card_list, [card], axis=0)
            turn_list = np.append(turn_list, [card_list], axis=0)
        match_list_X_f1.append(turn_list)

    for match in obj['X']:
        turn_list = np.empty(shape=(0,52))
        for turn in match:
            card_list = np.empty(shape=(0,))
            for card in turn[2]:
                card_list = np.append(card_list, [card], axis=0)
            turn_list = np.append(turn_list, [card_list], axis=0)
        match_list_X_f2.append(turn_list)

    for match in obj['X']:
        turn_list = np.empty(shape=(0,52))
        for turn in match:
            card_list = np.empty(shape=(0,))
            for card in turn[3]:
                card_list = np.append(card_list, [card], axis=0)
            turn_list = np.append(turn_list, [card_list], axis=0)
        match_list_X_f3.append(turn_list)

    for match in obj['Y']:
        turn_list = np.empty(shape=(0,52))
        for turn in match:
            card_list = np.empty(shape=(0,))
            for card in turn:
                card_list = np.append(card_list, [card], axis=0)
            turn_list = np.append(turn_list, [card_list], axis=0)
        match_list_Y.append(turn_list)

    return match_list_X_f0, match_list_X_f1, match_list_X_f2, match_list_X_f3, match_list_Y
    
def __import_data_pad__(filename: str):
    # Read json or dat
    
    with open(filename, 'r') as f:
        obj = json.load(f)

    # Set var
    match_list_X_f0 = []
    match_list_X_f1 = []
    match_list_X_f2 = []
    match_list_X_f3 = []

    match_list_Y = []

    for match in obj['X']:
        turn_list = []
        for turn in match:
            card_list = []
            for card in turn[0]:
                card_list.append(card)
            turn_list.append(card_list)
        match_list_X_f0.append(turn_list)

    for match in obj['X']:
        turn_list = []
        for turn in match:
            card_list = []
            for card in turn[1]:
                card_list.append(card)
            turn_list.append(card_list)
        match_list_X_f1.append(turn_list)

    for match in obj['X']:
        turn_list = []
        for turn in match:
            card_list = []
            for card in turn[2]:
                card_list.append(card)
            turn_list.append(card_list)
        match_list_X_f2.append(turn_list)

    for match in obj['X']:
        turn_list = []
        for turn in match:
            card_list = []
            for card in turn[3]:
                card_list.append(card)
            turn_list.append(card_list)
        match_list_X_f3.append(turn_list)

    for match in obj['Y']:
        turn_list = []
        for turn in match:
            card_list = []
            for card in turn:
                card_list.append(card)
            turn_list.append(card_list)
        match_list_Y.append(turn_list)

    return match_list_X_f0, match_list_X_f1, match_list_X_f2, match_list_X_f3, match_list_Y

def data_lstm_generator(match_list_X_f0, match_list_X_f1, match_list_X_f2, match_list_X_f3, match_list_Y, batch_size: int, n_match: int):
    r = 0
    while True:
        
        if r + batch_size >= n_match:
            r = 0

        max_length = get_max_length(match_list_X_f0[r: r+batch_size])

        x_train_1 = np.empty(shape=(batch_size, max_length, match_list_X_f0[0].shape[1]))
        x_train_2 = np.empty(shape=(batch_size, max_length, match_list_X_f0[0].shape[1]))
        x_train_3 = np.empty(shape=(batch_size, max_length, match_list_X_f0[0].shape[1]))
        x_train_4 = np.empty(shape=(batch_size, max_length, match_list_X_f0[0].shape[1]))

        y_train = np.empty(shape=(batch_size, max_length, match_list_Y[0].shape[1]))

        for i in range(0, batch_size):
            x_train_1[i] = pad_data(np.reshape(match_list_X_f0[r+i], (1, *match_list_X_f0[r+i].shape)), max_length)
            x_train_2[i] = pad_data(np.reshape(match_list_X_f1[r+i], (1, *match_list_X_f1[r+i].shape)), max_length)
            x_train_3[i] = pad_data(np.reshape(match_list_X_f2[r+i], (1, *match_list_X_f2[r+i].shape)), max_length)
            x_train_4[i] = pad_data(np.reshape(match_list_X_f3[r+i], (1, *match_list_X_f3[r+i].shape)), max_length)

            y_train[i] = pad_data(np.reshape(match_list_Y[r+i], (1, *match_list_Y[r+i].shape)), max_length)

        # assert x_train_1.shape == x_train_2.shape, "wrong data form"
        # assert x_train_2.shape == x_train_3.shape, "wrong data form"
        # assert x_train_3.shape == x_train_4.shape, "wrong data form"
        r += batch_size

        yield [x_train_1, x_train_2, x_train_3, x_train_4], y_train

def get_max_length(x):
    max_length = -1
    for match in x:
        if match.shape[0] > max_length:
            max_length = match.shape[0]

    return max_length

def dense_model():
    input = Input(shape=(208,), name = 'input')

    tensor = Dense(512, activation='relu', kernel_initializer='random_normal') (input)
    tensor = Dense(256, activation='relu', kernel_initializer='random_normal') (tensor)
    tensor = Dense(128, activation='relu', kernel_initializer='random_normal') (tensor)
    output = Dense(52, activation='sigmoid', kernel_initializer='random_normal') (tensor)

    model = Model(input, output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    model.summary()

    return model

def lstm_model_simple (input_max_length=None, config=None):
    
    input_op_pick = Input(shape=(input_max_length, 52), name="op_pick")
    op_pick_lstm = LSTM(64, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_op_pick)

    input_op_unpick = Input(shape=(input_max_length, 52), name="op_unpick")
    op_unpick_lstm = LSTM(64, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_op_unpick)

    input_op_discard = Input(shape=(input_max_length, 52), name="op_discard")
    op_discard_lstm = LSTM(64, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_op_discard)

    features = Concatenate(axis=1)([op_pick_lstm, op_unpick_lstm, op_discard_lstm])

    features = Dense(256, activation=relu, kernel_initializer='random_normal') (features)

    input_uncards = Input(shape=(input_max_length, 52), name="uncards")
    uncards_lstm = LSTM(64, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_uncards)

    uncards_dense = Dense(96, activation=relu, kernel_initializer='random_normal') (uncards_lstm)

    features = Concatenate(axis=1)([features, uncards_dense])

    features = Dense(512, activation=relu, kernel_initializer='random_normal') (features)

    features = Dense(128, activation=relu, kernel_initializer='random_normal') (features)

    output = Dense(52, activation='sigmoid', kernel_initializer='random_normal') (features)

    model = Model(inputs=[input_op_pick, input_op_unpick, input_op_discard, input_uncards], outputs=output)

    # adam = Adam(learning_rate=0.01)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    return model

def lstm_model (input_max_length=None, config=None):
    
    input_op_pick = Input(shape=(input_max_length, 52), name="op_pick")
    op_pick_lstm = LSTM(128, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_op_pick)

    input_op_unpick = Input(shape=(input_max_length, 52), name="op_unpick")
    op_unpick_lstm = LSTM(128, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_op_unpick)

    input_op_discard = Input(shape=(input_max_length, 52), name="op_discard")
    op_discard_lstm = LSTM(128, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_op_discard)

    input_uncards = Input(shape=(input_max_length, 52), name="uncards")
    uncards_lstm = LSTM(128, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_uncards)

    features = Concatenate(axis=1)([op_pick_lstm, op_unpick_lstm, op_discard_lstm, uncards_lstm])

    features = Dense(768, activation=relu, kernel_initializer='random_normal') (features)

    features = Dense(256, activation=relu, kernel_initializer='random_normal') (features)

    features = Dense(64, activation=relu, kernel_initializer='random_normal') (features)

    output = Dense(52, activation='sigmoid', kernel_initializer='random_normal') (features)

    model = Model(inputs=[input_op_pick, input_op_unpick, input_op_discard, input_uncards], outputs=output)

    # adam = Adam(learning_rate=0.01)

    model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    return model

def lstm_model_2 (input_max_length=None, config=None):
    
    input_op_pick = Input(shape=(input_max_length, 52), name="op_pick")
    op_pick_lstm = LSTM(128, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_op_pick)

    input_op_unpick = Input(shape=(input_max_length, 52), name="op_unpick")
    op_unpick_lstm = LSTM(128, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_op_unpick)

    input_op_discard = Input(shape=(input_max_length, 52), name="op_discard")
    op_discard_lstm = LSTM(128, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_op_discard)

    features = Concatenate(axis=1)([op_pick_lstm, op_unpick_lstm, op_discard_lstm])

    input_uncards = Input(shape=(input_max_length, 52), name="uncards")
    uncards_lstm = LSTM(128, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_uncards)

    features = Concatenate(axis=1)([features, uncards_lstm])

    features = Dense(256, activation=relu, kernel_initializer='random_normal') (features)

    # features = Dense(256, activation=relu, kernel_initializer='random_normal') (features)

    # features = Dense(64, activation=relu, kernel_initializer='random_normal') (features)

    output = Dense(52, activation='sigmoid', kernel_initializer='random_normal') (features)

    model = Model(inputs=[input_op_pick, input_op_unpick, input_op_discard, input_uncards], outputs=output)

    # adam = Adam(learning_rate=0.01)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    return model

def one_generator(x0, x1, x2, x3, y, n_match, seed=1):
    
    r = 0
    
    while True:

        r = r % n_match

        x_train_1 = np.reshape(x0[r], (1, x0[r].shape[0], x0[r].shape[1]))
        x_train_2 = np.reshape(x1[r], (1, x1[r].shape[0], x1[r].shape[1]))
        x_train_3 = np.reshape(x2[r], (1, x2[r].shape[0], x2[r].shape[1]))
        x_train_4 = np.reshape(x3[r], (1, x3[r].shape[0], x3[r].shape[1]))

        # Fix error in data
        if x_train_1.shape[1] == 0:
            r += seed
            continue

        y_train = np.reshape(y[r][y[r].shape[0] - 1], (1, y[r].shape[1]))

        assert x_train_1.shape == x_train_2.shape, "wrong data form"
        assert x_train_2.shape == x_train_3.shape, "wrong data form"
        assert x_train_3.shape == x_train_4.shape, "wrong data form"

        # assert x_train_1.shape == y_train.shape, "so wrong data form"
        r += seed
        yield [x_train_1, x_train_2, x_train_3, x_train_4], y_train

def pad_data (x, max_length):
    y = x
    for i, turn in enumerate(x):
        if turn.shape[0] < max_length:
            zeros = np.zeros(shape=(max_length - turn.shape[0], turn.shape[1]))
            y[i] = np.append(y[i], zeros, axis=0)

    return y



In [3]:
x0, x1, x2, x3, y = __import_data_lstm__("./dataset/output_200.json")
n_match = len(x0)

In [6]:


def __import_data_dense__(filename: str):
        # Read json or dat
    
    with open(filename, 'r') as f:
        obj = json.load(f)

    X_dense = np.empty(shape=(0, 208))
    Y_dense = np.empty(shape=(0, 52))

    for match in obj['X']:
        for turn in match:
            turn_vector = np.empty(shape=(0,))
            for feature in turn:
                for card in feature:
                    turn_vector = np.append(turn_vector, [card], axis=0)
            self.X_dense = np.append(self.X_dense, [turn_vector], axis=0)

    for match in obj['Y']:
        for turn in match:
            turn_vector = np.empty(shape=(0,))
            for card in feature:
                turn_vector = np.append(turn_vector, [card], axis=0)
            self.Y_dense = np.append(self.Y_dense, [turn_vector], axis=0)

    return X_dense, Y_dense


def __import_data_lstm__(filename: str):
    # Read json or dat
    
    with open(filename, 'r') as f:
        obj = json.load(f)

    # Set var
    match_list_X_f0 = []
    match_list_X_f1 = []
    match_list_X_f2 = []
    match_list_X_f3 = []

    match_list_Y = []

    for match in obj['X']:
        turn_list = np.empty(shape=(0,52))
        for turn in match:
            card_list = np.empty(shape=(0,))
            for card in turn[0]:
                card_list = np.append(card_list, [card], axis=0)
            turn_list = np.append(turn_list, [card_list], axis=0)
        match_list_X_f0.append(turn_list)

    for match in obj['X']:
        turn_list = np.empty(shape=(0,52))
        for turn in match:
            card_list = np.empty(shape=(0,))
            for card in turn[1]:
                card_list = np.append(card_list, [card], axis=0)
            turn_list = np.append(turn_list, [card_list], axis=0)
        match_list_X_f1.append(turn_list)

    for match in obj['X']:
        turn_list = np.empty(shape=(0,52))
        for turn in match:
            card_list = np.empty(shape=(0,))
            for card in turn[2]:
                card_list = np.append(card_list, [card], axis=0)
            turn_list = np.append(turn_list, [card_list], axis=0)
        match_list_X_f2.append(turn_list)

    for match in obj['X']:
        turn_list = np.empty(shape=(0,52))
        for turn in match:
            card_list = np.empty(shape=(0,))
            for card in turn[3]:
                card_list = np.append(card_list, [card], axis=0)
            turn_list = np.append(turn_list, [card_list], axis=0)
        match_list_X_f3.append(turn_list)

    for match in obj['Y']:
        turn_list = np.empty(shape=(0,52))
        for turn in match:
            card_list = np.empty(shape=(0,))
            for card in turn:
                card_list = np.append(card_list, [card], axis=0)
            turn_list = np.append(turn_list, [card_list], axis=0)
        match_list_Y.append(turn_list)

    return match_list_X_f0, match_list_X_f1, match_list_X_f2, match_list_X_f3, match_list_Y
    
def __import_data_pad__(filename: str):
    # Read json or dat
    
    with open(filename, 'r') as f:
        obj = json.load(f)

    # Set var
    match_list_X_f0 = []
    match_list_X_f1 = []
    match_list_X_f2 = []
    match_list_X_f3 = []

    match_list_Y = []

    for match in obj['X']:
        turn_list = []
        for turn in match:
            card_list = []
            for card in turn[0]:
                card_list.append(card)
            turn_list.append(card_list)
        match_list_X_f0.append(turn_list)

    for match in obj['X']:
        turn_list = []
        for turn in match:
            card_list = []
            for card in turn[1]:
                card_list.append(card)
            turn_list.append(card_list)
        match_list_X_f1.append(turn_list)

    for match in obj['X']:
        turn_list = []
        for turn in match:
            card_list = []
            for card in turn[2]:
                card_list.append(card)
            turn_list.append(card_list)
        match_list_X_f2.append(turn_list)

    for match in obj['X']:
        turn_list = []
        for turn in match:
            card_list = []
            for card in turn[3]:
                card_list.append(card)
            turn_list.append(card_list)
        match_list_X_f3.append(turn_list)

    for match in obj['Y']:
        turn_list = []
        for turn in match:
            card_list = []
            for card in turn:
                card_list.append(card)
            turn_list.append(card_list)
        match_list_Y.append(turn_list)

    return match_list_X_f0, match_list_X_f1, match_list_X_f2, match_list_X_f3, match_list_Y

def data_lstm_generator(match_list_X_f0, match_list_X_f1, match_list_X_f2, match_list_X_f3, match_list_Y, batch_size: int, n_match: int):
    r = 0
    while True:
        
        if r + batch_size >= n_match:
            r = 0

        max_length = get_max_length(match_list_X_f0[r: r+batch_size])

        x_train_1 = np.empty(shape=(batch_size, max_length, match_list_X_f0[0].shape[1]))
        x_train_2 = np.empty(shape=(batch_size, max_length, match_list_X_f0[0].shape[1]))
        x_train_3 = np.empty(shape=(batch_size, max_length, match_list_X_f0[0].shape[1]))
        x_train_4 = np.empty(shape=(batch_size, max_length, match_list_X_f0[0].shape[1]))

        y_train = np.empty(shape=(batch_size, max_length, match_list_Y[0].shape[1]))

        for i in range(0, batch_size):
            x_train_1[i] = pad_data(np.reshape(match_list_X_f0[r+i], (1, *match_list_X_f0[r+i].shape)), max_length)
            x_train_2[i] = pad_data(np.reshape(match_list_X_f1[r+i], (1, *match_list_X_f1[r+i].shape)), max_length)
            x_train_3[i] = pad_data(np.reshape(match_list_X_f2[r+i], (1, *match_list_X_f2[r+i].shape)), max_length)
            x_train_4[i] = pad_data(np.reshape(match_list_X_f3[r+i], (1, *match_list_X_f3[r+i].shape)), max_length)

            y_train[i] = pad_data(np.reshape(match_list_Y[r+i], (1, *match_list_Y[r+i].shape)), max_length)

        # assert x_train_1.shape == x_train_2.shape, "wrong data form"
        # assert x_train_2.shape == x_train_3.shape, "wrong data form"
        # assert x_train_3.shape == x_train_4.shape, "wrong data form"
        r += batch_size

        yield [x_train_1, x_train_2, x_train_3, x_train_4], y_train

def get_max_length(x):
    max_length = -1
    for match in x:
        if match.shape[0] > max_length:
            max_length = match.shape[0]

    return max_length

def dense_model():
    input = Input(shape=(208,), name = 'input')

    tensor = Dense(512, activation='relu', kernel_initializer='random_normal') (input)
    tensor = Dense(256, activation='relu', kernel_initializer='random_normal') (tensor)
    tensor = Dense(128, activation='relu', kernel_initializer='random_normal') (tensor)
    output = Dense(52, activation='sigmoid', kernel_initializer='random_normal') (tensor)

    model = Model(input, output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    model.summary()

    return model

def lstm_model_simple (input_max_length=None, config=None):
    
    input_op_pick = Input(shape=(input_max_length, 52), name="op_pick")
    op_pick_lstm = LSTM(64, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_op_pick)

    input_op_unpick = Input(shape=(input_max_length, 52), name="op_unpick")
    op_unpick_lstm = LSTM(64, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_op_unpick)

    input_op_discard = Input(shape=(input_max_length, 52), name="op_discard")
    op_discard_lstm = LSTM(64, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_op_discard)

    features = Concatenate(axis=1)([op_pick_lstm, op_unpick_lstm, op_discard_lstm])

    features = Dense(256, activation=relu, kernel_initializer='random_normal') (features)

    input_uncards = Input(shape=(input_max_length, 52), name="uncards")
    uncards_lstm = LSTM(64, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_uncards)

    uncards_dense = Dense(96, activation=relu, kernel_initializer='random_normal') (uncards_lstm)

    features = Concatenate(axis=1)([features, uncards_dense])

    features = Dense(512, activation=relu, kernel_initializer='random_normal') (features)

    features = Dense(128, activation=relu, kernel_initializer='random_normal') (features)

    output = Dense(52, activation='sigmoid', kernel_initializer='random_normal') (features)

    model = Model(inputs=[input_op_pick, input_op_unpick, input_op_discard, input_uncards], outputs=output)

    # adam = Adam(learning_rate=0.01)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    return model

def lstm_model (input_max_length=None, config=None):
    
    input_op_pick = Input(shape=(input_max_length, 52), name="op_pick")
    op_pick_lstm = LSTM(128, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_op_pick)

    input_op_unpick = Input(shape=(input_max_length, 52), name="op_unpick")
    op_unpick_lstm = LSTM(128, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_op_unpick)

    input_op_discard = Input(shape=(input_max_length, 52), name="op_discard")
    op_discard_lstm = LSTM(128, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_op_discard)

    input_uncards = Input(shape=(input_max_length, 52), name="uncards")
    uncards_lstm = LSTM(128, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_uncards)

    features = Concatenate(axis=1)([op_pick_lstm, op_unpick_lstm, op_discard_lstm, uncards_lstm])

    features = Dense(768, activation=relu, kernel_initializer='random_normal') (features)

    features = Dense(256, activation=relu, kernel_initializer='random_normal') (features)

    features = Dense(64, activation=relu, kernel_initializer='random_normal') (features)

    output = Dense(52, activation='sigmoid', kernel_initializer='random_normal') (features)

    model = Model(inputs=[input_op_pick, input_op_unpick, input_op_discard, input_uncards], outputs=output)

    # adam = Adam(learning_rate=0.01)

    model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    return model

def lstm_model_2 (input_max_length=None, config=None):
    
    input_op_pick = Input(shape=(input_max_length, 52), name="op_pick")
    op_pick_lstm = LSTM(64, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_op_pick)

    input_op_unpick = Input(shape=(input_max_length, 52), name="op_unpick")
    op_unpick_lstm = LSTM(64, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_op_unpick)

    input_op_discard = Input(shape=(input_max_length, 52), name="op_discard")
    op_discard_lstm = LSTM(64, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_op_discard)

    features = Concatenate(axis=1)([op_pick_lstm, op_unpick_lstm, op_discard_lstm])

    input_uncards = Input(shape=(input_max_length, 52), name="uncards")
    uncards_lstm = LSTM(64, return_sequences=False, return_state=False, kernel_initializer='random_normal') (input_uncards)

    features = Concatenate(axis=1)([features, uncards_lstm])

    features = Dense(256, activation=relu, kernel_initializer='random_normal') (features)

    # features = Dense(256, activation=relu, kernel_initializer='random_normal') (features)

    # features = Dense(64, activation=relu, kernel_initializer='random_normal') (features)

    output = Dense(52, activation='sigmoid', kernel_initializer='random_normal') (features)

    model = Model(inputs=[input_op_pick, input_op_unpick, input_op_discard, input_uncards], outputs=output)

    # adam = Adam(learning_rate=0.01)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    return model

def one_generator(x0, x1, x2, x3, y, n_match, seed=1):
    
    r = 0
    
    while True:

        r = r % n_match

        x_train_1 = np.reshape(x0[r], (1, x0[r].shape[0], x0[r].shape[1]))
        x_train_2 = np.reshape(x1[r], (1, x1[r].shape[0], x1[r].shape[1]))
        x_train_3 = np.reshape(x2[r], (1, x2[r].shape[0], x2[r].shape[1]))
        x_train_4 = np.reshape(x3[r], (1, x3[r].shape[0], x3[r].shape[1]))

        # Fix error in data
        if x_train_1.shape[1] == 0:
            r += seed
            continue

        y_train = np.reshape(y[r][y[r].shape[0] - 1], (1, y[r].shape[1]))

        assert x_train_1.shape == x_train_2.shape, "wrong data form"
        assert x_train_2.shape == x_train_3.shape, "wrong data form"
        assert x_train_3.shape == x_train_4.shape, "wrong data form"

        # assert x_train_1.shape == y_train.shape, "so wrong data form"
        r += seed
        yield [x_train_1, x_train_2, x_train_3, x_train_4], y_train

def pad_data (x, max_length):
    y = x
    for i, turn in enumerate(x):
        if turn.shape[0] < max_length:
            zeros = np.zeros(shape=(max_length - turn.shape[0], turn.shape[1]))
            y[i] = np.append(y[i], zeros, axis=0)

    return y



In [7]:

# model = lstm_model_simple()

# model = lstm_model()
model = lstm_model_2()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
op_pick (InputLayer)            [(None, None, 52)]   0                                            
__________________________________________________________________________________________________
op_unpick (InputLayer)          [(None, None, 52)]   0                                            
__________________________________________________________________________________________________
op_discard (InputLayer)         [(None, None, 52)]   0                                            
__________________________________________________________________________________________________
lstm_4 (LSTM)                   (None, 64)           29952       op_pick[0][0]                    
____________________________________________________________________________________________

In [8]:

# model.fit(x=one_generator(x0, x1, x2, x3, y, n_match, 0)[0], y=one_generator(x0, x1, x2, x3, y, n_match, 0)[1], epochs=1, verbose=1)

history = model.fit_generator(one_generator(x0, x1, x2, x3, y, n_match), steps_per_epoch=n_match, epochs=150,verbose=1)


Epoch 1/150
4074/4074 [==============================] - 28s 7ms/step - loss: 37.1472 - accuracy: 0.0523
Epoch 2/150
4074/4074 [==============================] - 28s 7ms/step - loss: 34.8944 - accuracy: 0.0368
Epoch 3/150
4074/4074 [==============================] - 30s 7ms/step - loss: 33.6654 - accuracy: 0.0363
Epoch 4/150
4074/4074 [==============================] - 30s 7ms/step - loss: 32.7869 - accuracy: 0.0361
Epoch 5/150
4074/4074 [==============================] - 30s 7ms/step - loss: 31.9813 - accuracy: 0.0400
Epoch 6/150
4074/4074 [==============================] - 30s 7ms/step - loss: 31.2338 - accuracy: 0.0388
Epoch 7/150
4074/4074 [==============================] - 30s 7ms/step - loss: 30.5549 - accuracy: 0.0314
Epoch 8/150
4074/4074 [==============================] - 30s 7ms/step - loss: 29.9204 - accuracy: 0.0326
Epoch 9/150
4074/4074 [==============================] - 30s 7ms/step - loss: 29.3594 - accuracy: 0.0324
Epoch 10/150
4074/4074 [==============================]

In [9]:

filename = 'lstm_simple_200'

with open('./history/' + filename + '_history.pkl', 'ab') as file_pi:
    pickle.dump(history.history, file_pi)

# np.save('./history/cdsc.npy', history.history)


In [10]:

# model.fit(x=one_generator(x0, x1, x2, x3, y, n_match, 0)[0], y=one_generator(x0, x1, x2, x3, y, n_match, 0)[1], epochs=1, verbose=1)

history = model.fit_generator(one_generator(x0, x1, x2, x3, y, n_match), steps_per_epoch=n_match, epochs=50,verbose=1)


Epoch 1/50
4074/4074 [==============================] - 28s 7ms/step - loss: 24.1619 - accuracy: 0.4936
Epoch 2/50
4074/4074 [==============================] - 28s 7ms/step - loss: 24.0914 - accuracy: 0.5017
Epoch 3/50
4074/4074 [==============================] - 28s 7ms/step - loss: 24.2363 - accuracy: 0.5069
Epoch 4/50
4074/4074 [==============================] - 29s 7ms/step - loss: 24.1635 - accuracy: 0.5211
Epoch 5/50
4074/4074 [==============================] - 30s 7ms/step - loss: 24.1919 - accuracy: 0.5258
Epoch 6/50
4074/4074 [==============================] - 30s 7ms/step - loss: 24.2463 - accuracy: 0.5304
Epoch 7/50
4074/4074 [==============================] - 31s 8ms/step - loss: 24.2304 - accuracy: 0.5439
Epoch 8/50
4074/4074 [==============================] - 31s 8ms/step - loss: 24.2503 - accuracy: 0.5282
Epoch 9/50
4074/4074 [==============================] - 31s 8ms/step - loss: 24.3433 - accuracy: 0.5535
Epoch 10/50
4074/4074 [==============================] - 31s 8ms

In [11]:

filename = 'lstm_simple_200'

with open('./history/' + filename + '_history.pkl', 'ab') as file_pi:
    pickle.dump(history.history, file_pi)

# np.save('./history/cdsc.npy', history.history)


In [12]:

model.save('lstm_200_200epoch.h5')  # save everything in HDF5 format
model_json = model.to_json()  # save just the config. replace with "to_yaml" for YAML serialization
with open("lstm_200_200epoch_config.json", "w") as f:
    f.write(model_json)
model.save_weights('lstm_200_200epoch_weights.h5') # save just the weights.
